In [7]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
from __future__ import print_function
from tensorflow import keras
import numpy as np
import cv2
import matplotlib.pyplot as plt
from preprocess import *
from utils import *

In [9]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Image loading and preprocessing


In [ ]:
crop_size = (256, 256)

def preprocess_generator(generator):
    while True:
        batch = next(generator)
        batch_crops = np.zeros((batch.shape[0], crop_size[0], crop_size[1], 3))
        for i in range(batch.shape[0]):
            batch_crops[i] = crop_img(batch[i], crop_size)
            batch_crops[i] = (batch_crops[i] - 0.5) * 2 # shift to -1,1

        assert not np.any(np.isnan(batch_crops))
        yield batch_crops, batch_crops


downscale_fact = 2
image_height_orig = 720
image_width_orig = 1280
image_height_ds = image_height_orig // downscale_fact
image_width_ds = image_width_orig // downscale_fact

train_ds, test_ds = create_dataflows('../images', (image_height_ds, image_width_ds), 16)

train_ds_prep = preprocess_generator(train_ds)
test_ds_prep = preprocess_generator(test_ds)

# train_ds_mapped = pair_mapping(train_ds)
# test_ds_mapped = pair_mapping(test_ds)

In [ ]:
plt.figure(figsize=(50, 50))
images = next(train_ds_prep)
for i, image in enumerate(images[0][:2]):
    ax = plt.subplot(5, 1, i + 1)
    plt.imshow(decenter_img(image))
    plt.axis("off")

train_ds.reset()

# The Model of the AutoEncoder

In [49]:
def ssim_loss(y_true, y_pred):
    # tf.debugging.assert_all_finite(y_true, "Data is not finite")
    # tf.debugging.assert_all_finite(y_pred, "Prediction is not finite")
    return 1 - tensorflow.reduce_mean(tensorflow.image.ssim_multiscale(decenter_img(y_true), decenter_img(y_pred), 1.0, filter_size=3))

def ssim_mae_loss(y_true, y_pred):
    # tf.debugging.assert_all_finite(y_true, "Data is not finite")
    # tf.debugging.assert_all_finite(y_pred, "Prediction is not finite")
    return ssim_loss(y_true, y_pred) * 0.5 + keras.losses.mae(y_true, y_pred) * 0.5

total_variation_weight = 1e-6

def tv_loss(y_true, y_pred):
    return ssim_loss(y_true, y_pred) + total_variation_weight * tf.reduce_sum(tf.image.total_variation(decenter_img(y_pred)))

In [103]:
from keras import backend as K, losses
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, ZeroPadding2D, BatchNormalization, Conv2DTranspose, \
    UpSampling2D, concatenate, DepthwiseConv2D
from keras.models import Model, Sequential

def create_block(model, layers):
    model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=1, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    model.add(Conv2D(layers, kernel_size=(1, 1), strides=1, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

def create_downscale_block(model, layers):
    model.add(DepthwiseConv2D(kernel_size=(2, 2), strides=2, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    model.add(Conv2D(layers, kernel_size=(1, 1), strides=1, kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

def create_upscale_block(model, layers):
    model.add(Conv2DTranspose(layers, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

def create_improved_baseline_model_12x_comp_old(image_size):
    ### Encoder ###

    model = Sequential()
    model.add(Conv2D(16, kernel_size=(1, 1), strides=1, kernel_initializer='he_normal',
                     padding='same', input_shape=(image_size[0], image_size[1], 3)))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    create_block(model, 32)
    create_block(model, 32)
    create_downscale_block(model, 32)

    create_block(model, 64)
    create_block(model, 64)
    create_downscale_block(model, 64)

    create_block(model, 64)
    create_block(model, 64)
    create_downscale_block(model, 32)

    create_block(model, 32)
    create_block(model, 32)
    create_downscale_block(model, 32)


    model.add(Activation('leaky_relu', name='encoded', dtype='float16'))


    ### Decoder ###

    model.add(Conv2DTranspose(64, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    create_block(model, 64)
    create_block(model, 64)

    model.add(Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    create_block(model, 32)
    create_block(model, 32)

    model.add(Conv2DTranspose(16, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    create_block(model, 16)
    create_block(model, 16)

    model.add(Conv2DTranspose(3, kernel_size=(3, 3), strides=2, padding='same', name='decoded', kernel_initializer='he_normal', activation='tanh'))

    print(("shape of encoded", K.int_shape(model.get_layer('encoded').output)))
    print(("shape of decoded", K.int_shape(model.get_layer('decoded').output)))

    return model

def create_improved_baseline_model_12x_comp(image_size):
    ### Encoder ###

    model = Sequential()
    model.add(Conv2D(16, kernel_size=(1, 1), strides=1, kernel_initializer='he_normal',
                     padding='same', input_shape=(image_size[0], image_size[1], 3)))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    create_block(model, 16)
    create_downscale_block(model, 32)

    create_block(model, 32)
    create_downscale_block(model, 32)

    create_block(model, 32)
    create_downscale_block(model, 32)

    create_block(model, 64)
    create_downscale_block(model, 64)
    create_block(model, 32)

    model.add(DepthwiseConv2D(kernel_size=(1, 1), strides=1, padding='same', kernel_initializer='he_normal'))
    model.add(Activation('leaky_relu', name='encoded', dtype='float16'))


    ### Decoder ###
    # model.add(UpSampling2D((2,2)))
    create_upscale_block(model, 64)
    create_block(model, 32)
    # create_block(model, 32)

    # model.add(UpSampling2D((2,2)))
    create_upscale_block(model, 32)
    # create_block(model, 32)
    # create_block(model, 32)

    create_upscale_block(model, 32)
    # create_block(model, 32)

    create_upscale_block(model, 16)
    create_block(model, 16)

    model.add(Conv2D(3, kernel_size=(1, 1), strides=1, padding='same', name='decoded', kernel_initializer='he_normal'))
    model.add(Activation('tanh'))

    # model.add(Conv2DTranspose(64, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    # # model.add(BatchNormalization())
    # model.add(Activation('leaky_relu'))
    #
    # create_block(model, 64)
    # create_block(model, 64)
    #
    # model.add(Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    # # model.add(BatchNormalization())
    # model.add(Activation('leaky_relu'))
    #
    # create_block(model, 32)
    # create_block(model, 32)
    #
    # model.add(Conv2DTranspose(16, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    # # model.add(BatchNormalization())
    # model.add(Activation('leaky_relu'))
    #
    # create_block(model, 16)
    # create_block(model, 16)
    #
    # model.add(Conv2DTranspose(3, kernel_size=(3, 3), strides=2, padding='same', name='decoded', kernel_initializer='he_normal', activation='tanh'))

    # print(("shape of encoded", K.int_shape(model.get_layer('encoded').output)))
    # print(("shape of decoded", K.int_shape(model.get_layer('decoded').output)))

    return model

def create_improved_baseline_model_12x_comp_new(image_size):
    ### Encoder ###

    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3), strides=(2, 2), kernel_initializer='he_normal',
                     padding='same', input_shape=(image_size[0], image_size[1], 3)))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    create_block(model, 16)
    create_block(model, 16)

    model.add(Conv2D(32, kernel_size=(3, 3), strides=(2, 2), kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    create_block(model, 32)

    model.add(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    model.add(Conv2D(32, kernel_size=(3, 3), strides=(2, 2), kernel_initializer='he_normal', padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu', name='encoded', dtype='float16'))


    ### Decoder ###

    model.add(Conv2DTranspose(64, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    model.add(Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    model.add(Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    model.add(Conv2DTranspose(16, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('leaky_relu'))

    model.add(Conv2D(3, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding='same', name='decoded'))

    print(("shape of encoded", K.int_shape(model.get_layer('encoded').output)))
    print(("shape of decoded", K.int_shape(model.get_layer('decoded').output)))

    return model

In [ ]:
autoencoder = create_improved_baseline_model_12x_comp_new(crop_size)
autoencoder.summary()

In [105]:
autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=ssim_loss,metrics=[ssim_loss, 'mae'])

# Training the AutoEncoder

In [ ]:
# Note: Delete the logs before running this. The command below should do this, but I would prefer to do it manually.
# !RMDIR "./logs/" /S /Q

# Launching Tensorboard
%tensorboard --logdir logs/fit --host localhost --port:6007
# localhost:6006 in browser

In [23]:
from keras.callbacks import CSVLogger, TensorBoard
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Set APPEND=TRUE if you are continuing the training, so that the log.csv wouldn't be reset!
csv_logger = CSVLogger('log.csv', append=False, separator=';')

In [ ]:
factor = 1

def train(epochs):
    autoencoder.fit(train_ds_prep,
                validation_data = test_ds_prep,
                steps_per_epoch = (train_ds.n*factor) // train_ds.batch_size,
                validation_steps = (test_ds.n*factor) // test_ds.batch_size,
                epochs=epochs,
                callbacks=[csv_logger, tensorboard_callback],
                verbose=1)

print("### Pretraining:")
autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='mae',metrics=[ssim_loss, 'mae'])
train(5)
print("### Training:")
autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0025), loss=ssim_loss,metrics=[ssim_loss, 'mae'])
train(60)

In [ ]:
# Saving the model:
# autoencoder.save('../model-saves/depthwise/')

In [18]:
# Loading the model:
# autoencoder = keras.models.load_model('../model-saves/improved-12x-RGB-v2/', custom_objects={
#     'ssim_loss': ssim_loss
# })

# autoencoder.summary()

# Running the AutoEncoder

In [ ]:
from matplotlib.pyplot import axes
import numpy as np

img_number = 4
test_images = next(test_ds_prep)[0][:img_number]
decoded_imgs = autoencoder.predict(test_images)

fig = plt.figure(figsize=(30, 30))

for i, in_img, out_img in zip(range(0, img_number, 1), test_images, decoded_imgs):
    plt.subplot(img_number, 3, i*3+1)
    plt.imshow(decenter_img(in_img))
    plt.subplot(img_number, 3, i*3 + 2)
    plt.imshow(decenter_img(out_img))
    plt.subplot(img_number, 3, i*3 + 3)
    rescaled_img = cv2.resize(decenter_img(in_img), (in_img.shape[0] // 3, in_img.shape[1] // 3))
    # manual_compression = tensorflow.cast(in_img*255. ,tensorflow.int32)
    # manual_compression = manual_compression - (manual_compression % 64)
    # manual_compression = tensorflow.cast(manual_compression ,tensorflow.float32)
    # manual_compression = tensorflow.cast(manual_compression/255. ,tensorflow.float32)
    plt.imshow(rescaled_img)
    # print(f'Image({i}) ms-ssim={ssim_loss(in_img, out_img):5.4f}, mae={np.mean(np.abs(in_img - out_img)):5.4f}, ms-ssim(manual)={ssim_loss(in_img, manual_compression):5.4f}, mae(manual)={np.mean(np.abs(in_img - manual_compression)):5.4f}')